# Scripts Mosaicos

In [1]:
import ee
import geemap
import os
import folium

In [2]:
Map = geemap.Map()
Map2 = geemap.Map()
Map3 = geemap.Map()

## Área de Mosaicos

### Patagonia

In [3]:
work_dir = os.path.expanduser('../../Data/shapes/')
patagonia = os.path.join(work_dir, 'patagonia_nueva.shp')
patagonia_grande = geemap.shp_to_ee(patagonia)
Map.addLayer(patagonia_grande, {}, 'Patagonia')
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [4]:
work_dir = os.path.expanduser('../../Data/shapes/')
norte = os.path.join(work_dir, 'area_norte.shp')
patagonia_norte = geemap.shp_to_ee(norte)
Map.addLayer(patagonia_norte, {}, 'norte')

In [5]:
work_dir = os.path.expanduser('../../Data/shapes/')
centro = os.path.join(work_dir, 'area_centro.shp')
patagonia_centro = geemap.shp_to_ee(centro)
Map.addLayer(patagonia_centro, {}, 'centro')

In [6]:
work_dir = os.path.expanduser('../../Data/shapes/')
sur = os.path.join(work_dir, 'area_sur.shp')
patagonia_sur = geemap.shp_to_ee(sur)
Map.addLayer(patagonia_sur, {}, 'sur')

## Funciones

In [7]:
def addNDVI(image): 
    ndvi = image.normalizedDifference(['B4', 'B3']).rename('NDVI')
    return image.addBands(ndvi)

In [8]:
def addNDWI(image):
    ndwi = image.normalizedDifference(['B2','B4']).rename('NDWI')
    return image.addBands(ndwi)

In [9]:
def addNDGI(image):
    ndgi = image.normalizedDifference(['B2','B3']).rename('NDGI')
    return image.addBands(ndgi)

In [10]:
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

In [11]:
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [12]:
def Cloud50(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(50))  

In [13]:
def Cloud30(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(30)) 

In [14]:
def Agua(image):
    return image.updateMask(ndwi.lte(-0.1))

In [15]:
def Agua1986(image):
    return image.updateMask(ndwi1986.gte(0.1))

In [16]:
def Agua1985(image):
    return image.updateMask(ndwi1985.gte(0.1))

In [17]:
def Agua1984(image):
    return image.updateMask(ndwi1984.gte(0.1))

In [18]:
def Ndvi1986(image):
    return image.updateMask(ndvi1986.gt(0.05))

In [19]:
def Ndvi1985(image):
    return image.updateMask(ndvi1985.lt(0.04))

In [20]:
def Ndvi1984(image):
    return image.updateMask(ndvi1984.lt(0.2))

In [21]:
def Ndvi2(image):
    return image.updateMask(ndvi.gte(0.2))

### Funciones SUR

In [22]:
def Ndvi1986_sur(image):
    return image.updateMask(ndvisur.gte(0.05))

In [23]:
def Agua1986_sur(image):
    return image.updateMask(ndwisur.gt(0.4))

## Colecciones

### Verano 1986

In [24]:
start_date = '1986-10-01'
end_date = '1987-03-31'

l5_1986 = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [25]:
vizParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 0.5,
  'gamma': [0.95, 1.1, 1]
}

In [26]:
coleccion1986 = l5_1986.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate)

In [27]:
start_date = '1986-10-01'
end_date = '1987-03-31'

l4_1986 = ee.ImageCollection('LANDSAT/LT04/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [28]:
vizParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 0.5,
  'gamma': [0.95, 1.1, 1]
}

In [29]:
coleccion1986_l4 = l4_1986.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate)

In [30]:
median = l5_1986.median()

visParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 0.4,
}

Map.addLayer(median, visParams, 'Median')
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

#### Verano Sur 1984-1987

In [31]:
start_date = '1984-01-01'
end_date = '1989-03-31'

l5_1986_sur = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [32]:
vizParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 0.5,
  'gamma': [0.95, 1.1, 1]
}

In [33]:
coleccionsur = l5_1986_sur.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate)

In [34]:
start_date = '1983-10-01'
end_date = '1984-03-31'

l4_1986_sur = ee.ImageCollection('LANDSAT/LT04/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [35]:
vizParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 0.5,
  'gamma': [0.95, 1.1, 1]
}

In [36]:
coleccionsur_l4 = l4_1986_sur.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate)

In [37]:
start_date = '1982-10-01'
end_date = '1990-03-31'

l4_1990_sur = ee.ImageCollection('LANDSAT/LT04/C01/T1_TOA') \
    .filterBounds(patagonia_sur) \
    .filterDate(start_date, end_date)

In [38]:
coleccionsur_l4_1990 = l4_1990_sur.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate)

In [39]:
median_sur = l5_1986_sur.median()

visParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 0.4,
}

Map.addLayer(median_sur, visParams, 'Median')
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

### Verano 1985

In [40]:
start_date = '1985-10-01'
end_date = '1986-03-31'

l5_1985 = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [41]:
coleccion1985 = l5_1985.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate)

In [42]:
start_date = '1985-10-01'
end_date = '1986-03-31'

l4_1985 = ee.ImageCollection('LANDSAT/LT04/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [43]:
coleccion1985_l4 = l4_1985.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate)

In [44]:
median_1985 = l5_1985.median()

visParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 0.4,
}

Map.addLayer(median_1985, visParams, 'Median 1985')
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

### Verano 1984

In [45]:
start_date = '1984-10-01'
end_date = '1985-03-31'

l5_1984 = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [46]:
coleccion1984 = l5_1984.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate)

In [47]:
start_date = '1984-10-01'
end_date = '1985-03-31'

l4_1984 = ee.ImageCollection('LANDSAT/LT04/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [48]:
coleccion1984_l4 = l4_1984.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate)

In [49]:
median_1984 = l5_1984.median()

visParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 0.4,
}

Map.addLayer(median_1984, visParams, 'Median 1984')
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

### Verano 1983

In [50]:
start_date = '1984-01-01'
end_date = '1984-10-01'

l5_1983 = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [51]:
coleccion1983 = l5_1983.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate)

## Quality Mosaic NDVI

In [52]:
NDVI_1986 = coleccion1986.qualityMosaic('NDVI')
NDVI_1985 = coleccion1985.qualityMosaic('NDVI')
NDVI_1984 = coleccion1984.qualityMosaic('NDVI')
NDVI_1983 = coleccion1983.qualityMosaic('NDVI')

In [53]:
NDVI_1986_l4 = coleccion1986_l4.qualityMosaic('NDVI')
NDVI_1985_l4 = coleccion1985_l4.qualityMosaic('NDVI')
NDVI_1984_l4 = coleccion1984_l4.qualityMosaic('NDVI')
NDVI_1990_l4 = coleccionsur_l4_1990.qualityMosaic('NDVI')

In [54]:
NDVI_sur = coleccionsur.qualityMosaic('NDVI')
NDVI_sur_l4 = coleccionsur_l4.qualityMosaic('NDVI')

In [55]:
visParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 0.4,
    'gamma': [0.95, 1.1, 1],
}

Map.addLayer(NDVI_1986, visParams, 'Greenest 1986')
Map.addLayer(NDVI_1985, visParams, 'Greenest 1985')
Map.addLayer(NDVI_1984, visParams, 'Greenest 1984')
Map.addLayer(NDVI_1983, visParams, 'Greenest 1983')
Map.addLayer(NDVI_sur, visParams, 'Greenest sur')
Map.centerObject(patagonia_grande, 6)
Map

Map(center=[-47.89147413528022, -68.82549165231002], controls=(WidgetControl(options=['position', 'transparent…

In [56]:
Map.addLayer(NDVI_1986, visParams, 'Greenest 1986 L4')
Map.addLayer(NDVI_1985, visParams, 'Greenest 1985 L4')
Map.addLayer(NDVI_1984, visParams, 'Greenest 1984 L4')
Map.addLayer(NDVI_1983, visParams, 'Greenest 1983 L4')
Map.addLayer(NDVI_1990_l4, visParams, 'Greenest 1990 L4')

In [57]:
ndvi1986 = NDVI_1986.select('NDVI')
ndvi1985 = NDVI_1985.select('NDVI')
ndvi1984 = NDVI_1984.select('NDVI')
ndvi1983 = NDVI_1983.select('NDVI')
ndvisur = NDVI_sur.select('NDVI')
palette = ['#d73027', '#f46d43', '#fdae61', '#fee08b', '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850']

In [58]:
Map.addLayer(ndvi1986, {'palette': palette}, 'NDVI_1986')
Map.addLayer(ndvi1985, {'palette': palette}, 'NDVI_1985')
Map.addLayer(ndvi1984, {'palette': palette}, 'NDVI_1984')
Map.addLayer(ndvi1983, {'palette': palette}, 'NDVI_1983')
Map.addLayer(ndvisur,{'palette': palette}, 'NDVI_sur')
Map

Map(center=[-47.89147413528022, -68.82549165231002], controls=(WidgetControl(options=['position', 'transparent…

## Agua

### Verano 1986

In [59]:
aquast_1986 = coleccion1986.qualityMosaic('NDWI')
aquast_sur = coleccionsur.qualityMosaic('NDWI')

### Verano 1985

In [60]:
aquast_1985 = coleccion1985.qualityMosaic('NDWI')

### Verano 1984

In [61]:
aquast_1984 = coleccion1984.qualityMosaic('NDWI')

### Quality Mosaic NDWI

In [62]:
visParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 0.4,
    'gamma': [0.95, 1.1, 1],
}

Map2.addLayer(aquast_1986, visParams, 'Agua 1986')
Map2.addLayer(aquast_1985, visParams, 'Agua 1985')
Map2.addLayer(aquast_1984, visParams, 'Agua 1984')
Map2.addLayer(aquast_sur, visParams, 'Agua sur')
Map2.centerObject(patagonia_grande, 6)
Map2

Map(center=[-47.89147413528022, -68.82549165231002], controls=(WidgetControl(options=['position', 'transparent…

In [63]:
ndwi1986 = aquast_1986.select('NDWI')
ndwi_viz = ['00FFFF', '0000FF']
ndwi1985 = aquast_1985.select('NDWI')
ndwi1984 = aquast_1984.select('NDWI')
ndwisur = aquast_sur.select('NDWI')
Map2.addLayer(ndwi1986, {'palette': ndwi_viz}, 'NDWI_1986')
Map2.addLayer(ndwi1985, {'palette': ndwi_viz}, 'NDWI_1985')
Map2.addLayer(ndwi1984, {'palette': ndwi_viz}, 'NDWI_1984')
Map2.addLayer(ndwisur, {'palette': ndwi_viz}, 'NDWI_sur')
Map2

Map(center=[-47.89147413528022, -68.82549165231002], controls=(WidgetControl(options=['position', 'transparent…

In [64]:
ndwi_rgb = ndwi1986.visualize(**{
    'min': 0.5,
    'max':1,
    'palette': ['00FFFF', '0000FF']
})

In [65]:
ndwi_mask = ndwi1986.updateMask(ndwi1986.gte(0.4))

In [66]:
map_ndwi_mask = folium.Map(location = [-45, -73], zoom_start = 5)
map_ndwi_mask.add_ee_layer(ndwi_mask, {} , 'NDWI sin tierra')
display(map_ndwi_mask)

In [67]:
ndwi_mask1986 = ndwi1986.updateMask(ndwi1986.gte(0.4))

In [68]:
map_ndwi_mask1986 = folium.Map(location = [-45, -73], zoom_start = 5)
map_ndwi_mask1986.add_ee_layer(ndwi_mask1986, {}, 'NDWI sin tierra')
display(map_ndwi_mask1986)

## Máscaras

### Verano 1986

#### Nubes 50

In [69]:
coleccion86 = coleccion1986.map(Cloud50)
greenest86 = coleccion86.qualityMosaic('NDVI')
map_1986 = folium.Map(location = [-45, -73], zoom_start = 5)
map_1986.add_ee_layer(greenest86, vizParams, 'Mapa sin Nubes')
display(map_1986)

#### Nubes 30

In [70]:
coleccion86 = coleccion1986.map(Cloud30)
greenest86 = coleccion86.qualityMosaic('NDVI')
map_1986 = folium.Map(location = [-45, -73], zoom_start = 5)
map_1986.add_ee_layer(greenest86, vizParams, 'Mapa sin Nubes')
display(map_1986)

#### Nubes Sur

In [71]:
coleccionsur_ = coleccionsur.map(Cloud30)
coleccionsur_ = coleccionsur_.map(Ndvi1986_sur)
greenestsur_ = coleccionsur_.qualityMosaic('NDVI')
map_sur = folium.Map(location = [-45, -73], zoom_start = 5)
map_sur.add_ee_layer(greenestsur_, vizParams, 'Mapa sin Nubes')
display(map_sur)

In [72]:
coleccionsur2 = coleccionsur.map(Cloud50)
coleccionsur2 = coleccionsur2.map(Ndvi1986_sur)
greenestsur2 = coleccionsur2.qualityMosaic('NDVI')
map_sur2 = folium.Map(location = [-45, -73], zoom_start = 5)
map_sur2.add_ee_layer(greenestsur2, vizParams, 'Mapa sin Nubes')
display(map_sur2)

#### Filtro de agua

In [73]:
coleccion86_ = coleccion1986.map(Cloud50)
coleccion86_ = coleccion86_.map(Agua1986_sur)
greenest86_ = coleccion86_.qualityMosaic('NDWI')
map_1986_ = folium.Map(location = [-45, -73], zoom_start = 5)
map_1986_.add_ee_layer(greenest86_, vizParams, 'Mapa sin Nubes')
display(map_1986_)

In [74]:
agua_1986 = coleccion1986.map(Cloud50)
agua_1986 = agua_1986.map(Agua1986)
aquast86_ = agua_1986.qualityMosaic('NDWI')
map_agua1986 = folium.Map(location = [-45, -73], zoom_start = 5)
map_agua1986.add_ee_layer(aquast86_, vizParams, 'Mapa sin Nubes')
display(map_agua1986)

In [75]:
coleccion86 = coleccion1986.map(Cloud50)
greenest86 = coleccion86.qualityMosaic('NDVI')
map_1986 = folium.Map(location = [-45, -73], zoom_start = 5)
map_1986.add_ee_layer(greenest86, vizParams, 'Mapa sin Nubes')
display(map_1986)

#### Filtro NDVI

In [76]:
coleccion86 = coleccion1986.map(Cloud50)
coleccion86 = coleccion86.map(Ndvi1986)
greenest86_2 = coleccion86.qualityMosaic('NDVI')
map_1986_2 = folium.Map(location = [-45, -73], zoom_start = 5)
map_1986_2.add_ee_layer(greenest86_2, vizParams, 'Mapa sin Nubes')
display(map_1986_2)

In [77]:
coleccion_sur2 = coleccionsur.map(Cloud30)
coleccion_sur2 = coleccion_sur2.map(Ndvi1986_sur)
coleccion_sur2 = coleccion_sur2.map(Agua1986_sur)
greenest_sur2 = coleccion_sur2.qualityMosaic('NDVI')
map_sur2 = folium.Map(location = [-45, -73], zoom_start = 5)
map_sur2.add_ee_layer(greenest_sur2, vizParams, 'Mapa sin Nubes')
display(map_sur2)

### Verano 1985

In [78]:
coleccion85 = coleccion1985.map(Cloud50)
# coleccion2020 = coleccion2020.map(Agua)
greenest85 = coleccion85.qualityMosaic('NDVI')
map_1985 = folium.Map(location = [-45, -73], zoom_start = 5)
map_1985.add_ee_layer(greenest85, vizParams, 'Mapa sin nubes')
display(map_1985)

In [79]:
agua_1985 = coleccionsur.map(Agua1986_sur)
aquast85_ = agua_1985.qualityMosaic('NDWI')
map_agua1985 = folium.Map(location = [-45, -73], zoom_start = 5)
map_agua1985.add_ee_layer(aquast85_, vizParams, 'Mapa sin Nubes')
display(map_agua1985)

### Verano 1984

In [80]:
coleccion84 = coleccion1984.map(Cloud50)
# coleccion2019 = coleccion2019.map(Agua)
greenest84 = coleccion84.qualityMosaic('NDVI')
map_1984 = folium.Map(location = [-45, -73], zoom_start = 5)
map_1984.add_ee_layer(greenest84, vizParams, 'Mapa sin nubes')
display(map_1984)

In [81]:
coleccion84 = coleccion1984.map(Cloud30)
# coleccion2019 = coleccion2019.map(Agua)
greenest84 = coleccion84.qualityMosaic('NDVI')
map_1984 = folium.Map(location = [-45, -73], zoom_start = 5)
map_1984.add_ee_layer(greenest84, vizParams, 'Mapa sin nubes')
display(map_1984)

## Mosaicos

In [82]:
Map3.addLayer(ndvi1986, {'palette': palette}, 'NDVI_1986')
Map3

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [83]:
mosaic = ee.ImageCollection([greenestsur_,greenest86]).mosaic()

In [84]:
Map3.addLayer(mosaic, vizParams, 'Mosaico')
Map3

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [85]:
mosaic2 = ee.ImageCollection([mosaic, aquast85_]).mosaic()

In [86]:
Map3.addLayer(mosaic2, vizParams, 'Mosaico2')
Map3

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [87]:
mosaic3 = ee.ImageCollection([mosaic2, greenest86_2]).mosaic()

In [88]:
Map3.addLayer(mosaic3, vizParams, 'Mosaico3')
Map3

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [89]:
mosaic4 = ee.ImageCollection([mosaic3, greenestsur_]).mosaic()

In [90]:
Map3.addLayer(mosaic4, vizParams, 'Mosaico 4')
Map3

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [91]:
mosaic5 = ee.ImageCollection([mosaic4,aquast85_]).mosaic()

In [92]:
Map3.addLayer(mosaic5, vizParams, 'Mosaico 5')
Map3

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [93]:
mosaic6 = ee.ImageCollection([mosaic5,greenestsur_]).mosaic()

In [94]:
Map3.addLayer(mosaic6, vizParams, 'Mosaico 6')
Map3

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…